In [ ]:
import pandas as pd
import osmnx as ox
import numpy as np
import json

In [ ]:
cities_pl = ['Warszawa', 'Kraków', 'Wrocław', 'Poznań', 'Szczecin']

all_nodes_pl = pd.DataFrame()
all_edges_pl = pd.DataFrame()

for city in cities_pl:
    G =ox.graph.graph_from_place(f"{city}, Poland", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_pl = pd.concat([all_nodes_pl, gdf_nodes])
    all_edges_pl = pd.concat([all_edges_pl, gdf_edges])


In [ ]:
cities_usa = ['Seattle', 'New York City']

all_nodes_usa = pd.DataFrame()
all_edges_usa = pd.DataFrame()

for city in cities_usa:
    G =ox.graph.graph_from_place(f"{city}, United States", network_type="drive", simplify=False)
    gdf_nodes, gdf_edges = ox.graph_to_gdfs(G)
    all_nodes_usa = pd.concat([all_nodes_usa, gdf_nodes])
    all_edges_usa = pd.concat([all_edges_usa, gdf_edges])

In [ ]:
all_attributes = {}
for col in ['highway', 'access', 'junction', 'bridge', 'tunnel']:
    edges_usa = all_edges_usa[col].dropna().unique()
    edges_pl = all_edges_pl[col].dropna().unique()
    all_edges = set(np.concatenate((edges_pl, edges_usa)))
    all_attributes[f'edges_{col}'] = list(all_edges)

     
nodes_highway_usa = all_nodes_usa['highway'].dropna().unique()
nodes_highway_pl = all_nodes_pl['highway'].dropna().unique()   
all_nodes_highway = set(np.concatenate((nodes_highway_pl, nodes_highway_usa)))
all_attributes['nodes_highway'] = list(all_nodes_highway)

with open('../data/osmnx_attributes.json', 'w') as f:
    json.dump(all_attributes, f)
    

In [ ]:
# Assuming list1 and list2 are your two lists
set([item for item in all_nodes_highway if item not in all_nodes_usa['highway'].dropna().unique()])